# Подготовка текста (6 баллов)


В первой части нужно подготовить данные

In [23]:
!pip install --quiet sentencepiece datasets transformers


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [24]:
import random
import torch
import numpy as np
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
import os

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

Берем текст из архива data.tar.gz

In [25]:
!tar -xzvf data.tar.gz

x books_txt/
x books_txt/KRYLOW/
x books_txt/ZHUKOWSKIJ/
x books_txt/PISAREW/
x books_txt/DAWYDOW/
x books_txt/NIKITIN_I/
x books_txt/PISEMSKIJ/
x books_txt/SOMOW/
x books_txt/.DS_Store
x books_txt/GERCEN/
x books_txt/OSTROWSKIJ/
x books_txt/LERMONTOW/
x books_txt/DOWLATOW/
x books_txt/GOGOL/
x books_txt/KUPRIN/
x books_txt/TOLSTOJ_A_K/
x books_txt/LEWITOW/
x books_txt/BULGAKOW/
x books_txt/BARATYNSKIJ/
x books_txt/DOSTOEWSKIJ/
x books_txt/POGORELSKIJ/
x books_txt/TYNYANOW/
x books_txt/GRIGORIEW/
x books_txt/FONWIZIN/
x books_txt/SALTYKOWSHEDRIN/
x books_txt/ODOEWSKIJ/
x books_txt/GONCHAROW/
x books_txt/LESKOW/
x books_txt/CHEHOW/
x books_txt/TOLSTOJ/
x books_txt/BELINSKIJ/
x books_txt/NAREZHNYJ/
x books_txt/GARSHIN/
x books_txt/ERSHOW/
x books_txt/HARMS/
x books_txt/POMQLOWSKIJ/
x books_txt/REPIN/
x books_txt/NEKRASOW/
x books_txt/DOBROLYUBOW/
x books_txt/KARAMZIN/
x books_txt/SUMAROKOW/
x books_txt/PRUTKOW/
x books_txt/TURGENEW/
x books_txt/TUTCHEW/
x books_txt/GRIBOEDOW/
x books_txt

In [26]:
# реализуйте функцию, которя читает файл
def read_file(path):
    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

In [27]:
paths = ['books_txt/DOSTOEWSKIJ/malenkiy.txt']

In [28]:
read_file(paths[0])

'Федор Михайлович Достоевский. Маленький герой\n\n\nФедор Михайлович Достоевский. Маленький герой\n\n\n\n     Версия 1.0 от  02  января  1998  г.  Сверка  произведена  по  "Собранию\nсочинений в десяти томах" (Москва, Художественная литература, 1956-57).\n\n     Права на  этот  электронный  текст  принадлежат  Публичной  электронной\nбиблиотеке (Евгению Пескину), 1998 год. Разрешено свободное  распространение\nпри условии  сохранения  целостности  текста  (включая  данную  информацию).\nРазрешено свободное  использование  для  некоммерческих  целей  при  условии\nссылки на источник. Публичная Электронная Библиотека - товарный знак и  знак\nобслуживания, принадлежащие Евгению Пескину.\n     E-mail:eugene@eugene.msk.su\n     \nWWW: http://www.online.ru/sp/eel/russian\n\n     -----------------------------------------------------------------------\n\n\nМАЛЕНЬКИЙ ГЕРОЙ \n\n                          Из неизвестных мемуаров\n\n\n     Было мне тогда без  малого  одиннадцать  лет.  В  июле  отп

In [29]:
def remove_consecutive_spaces(string):
    return ' '.join(string.split())


def chars(f, t):
    return list(map(chr, range(ord(f), ord(t) + 1)))


valid_chars = set(chars('а', 'я') + ['ё'] + chars('a', 'z') + chars('0', '9')
                  + list("<>(){}" + ".,\"!?;:-*—\'"))


# Приведите текст к нижнему регистру
# yберите лишние символы и пробелы идущие подряд
def clean_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = ''.join([char if char in valid_chars else ' ' for char in text])
    text = remove_consecutive_spaces(text)
    return text

In [30]:
clean_text(read_file(paths[0]))

'федор михайлович достоевский. маленький герой федор михайлович достоевский. маленький герой версия 1.0 от 02 января 1998 г. сверка произведена по "собранию сочинений в десяти томах" (москва, художественная литература, 1956-57). права на этот электронный текст принадлежат публичной электронной библиотеке (евгению пескину), 1998 год. разрешено свободное распространение при условии сохранения целостности текста (включая данную информацию). разрешено свободное использование для некоммерческих целей при условии ссылки на источник. публичная электронная библиотека - товарный знак и знак обслуживания, принадлежащие евгению пескину. e-mail:eugene eugene.msk.su www: http: www.online.ru sp eel russian ----------------------------------------------------------------------- маленький герой из неизвестных мемуаров было мне тогда без малого одиннадцать лет. в июле отпустили меня гостить в подмосковную деревню, к моему родственнику, т-ву, к которому в то время съехалось человек пятьдесят, а может бы

Напшите код, который по списку файлов читает тексты и нормализует их.

In [31]:
def load_texts(paths):
    texts = [clean_text(read_file(path)) for path in paths]

    return texts

In [32]:
def get_all_files(root):
  files_paths = []
  for root, dir, files in os.walk(root):
    for file in files:
      if file.endswith(".txt"):
          files_paths.append(os.path.join(root, file))
  return files_paths

In [38]:
paths = get_all_files("books_txt")
paths.sort()
print(f'files cnt: {len(paths)}')

files cnt: 749


In [39]:
# Выделим небольшой процент данных для теста
paths_train, paths_test = train_test_split(paths, test_size=0.03, random_state=12345)

texts_train = load_texts(paths_train)
texts_test = load_texts(paths_test)

In [40]:
paths_test

['books_txt/CHEHOW/r_knyaginya.txt',
 'books_txt/CHEHOW/ostrova.txt',
 'books_txt/CHEHOW/about_kireev.txt',
 'books_txt/SALTYKOWSHEDRIN/saltykov8_2.txt',
 'books_txt/ODOEWSKIJ/od_kosmo.txt',
 'books_txt/LESKOW/yazv.txt',
 'books_txt/CHEHOW/mylife.txt',
 'books_txt/LESKOW/pavlin.txt',
 'books_txt/DOSTOEWSKIJ/zametki.txt',
 'books_txt/SALTYKOWSHEDRIN/s_tale02.txt',
 'books_txt/TOLSTOJ/junost.txt',
 'books_txt/OSTROWSKIJ/money.txt',
 'books_txt/LESKOW/leskov6_4.txt',
 'books_txt/DOSTOEWSKIJ/verbas.txt',
 'books_txt/TOLSTOJ/s_will.txt',
 'books_txt/CHEHOW/krasavitsy.txt',
 'books_txt/CHEHOW/happiness.txt',
 'books_txt/CHEHOW/reading.txt',
 'books_txt/SALTYKOWSHEDRIN/s_tale03.txt',
 'books_txt/SOMOW/about.txt',
 'books_txt/TOLSTOJ/s_molitwa.txt',
 'books_txt/GRIGORIEW/grigoriev1_4.txt',
 'books_txt/SALTYKOWSHEDRIN/s_tale04.txt']

Для токенизации используется готовое решение из библиотеки Huggingface

In [41]:
from transformers import GPT2TokenizerFast
from tokenizers.decoders import ByteLevel, WordPiece
from transformers import BertTokenizerFast

decoder = WordPiece()

/Users/ddulaev/PycharmProjects/dltheory/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [42]:
tokenizer = BertTokenizerFast('vocab.txt')

In [43]:
pad_token_idx = 0
sep_token_idx = 3

In [44]:
def encode(text):
    return tokenizer.encode(text, add_special_tokens=False)

In [47]:
ids = encode("я помню чудное мгновенье")
ids

[156, 2769, 2723, 539, 2646, 1881]

In [48]:
tokenizer.convert_ids_to_tokens(ids)

['я', 'помню', 'чуд', '##ное', 'мгнов', '##енье']

In [49]:
decoder.decode(tokenizer.convert_ids_to_tokens(ids))

'я помню чудное мгновенье'



Конкатенируйте все тексты в большую последовательность токенов


In [53]:
def combine_texts(texts):
    tokens = [token for text in texts for token in encode(text)]

    return tokens

In [54]:
tokenized_train = combine_texts(texts_train)

Посмотрим количество токенов, должно получиться примерно 35087577

In [55]:
print(len(tokenized_train))

35054756


In [56]:
max_seq_len = 256

batch_size = 16

Далее нужно нарезать текст на части размером `max_seq_len + 1`. Полезно что бы части пересекались, для этого добавлен параметр `offset`

In [57]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            texts,
            offset,
            max_seq_len,
    ):
        self.texts = texts
        self.offset = offset
        self.max_seq_len = max_seq_len
        self.tokens = combine_texts(texts)
        self.slices = []
        for i in range(0, len(self.tokens) - max_seq_len, offset):
            self.slices.append(self.tokens[i:i + max_seq_len + 1])

    def __getitem__(self, idx):
        return self.slices[idx]

    def __len__(self):
        return len(self.slices)

In [58]:
def collate_batch(batch):
    src_list = []
    for src in batch:
        src_list.append(torch.tensor(src))

    return pad_sequence(src_list, padding_value=pad_token_idx, batch_first=True)

In [60]:
train_data = TextDataset(
    texts_train,
    max_seq_len // 2,
    max_seq_len=max_seq_len,
)
train_dataloader = DataLoader(train_data, batch_size, shuffle=True, collate_fn=collate_batch)

In [61]:
test_data = TextDataset(
    texts_test,
    max_seq_len * 2,
    max_seq_len=max_seq_len,
)  # Что бы не ждать будем использовать большой offset
test_dataloader = DataLoader(test_data, batch_size, collate_fn=collate_batch)

In [62]:
print(len(train_data))

273864


In [63]:
batch = next(iter(train_dataloader))
print(batch.shape)

torch.Size([16, 257])


Ожидается размер torch. `torch.Size([16, 257])`
